In [7]:
import json
import re

def parse_enums(file_path):
    with open(file_path, 'r', encoding="utf8") as f:
        content = f.read()
    enum_matches = re.finditer(r'export enum (\w+) \{([\s\S]*?)\}', content)
    enums = {}
    for match in enum_matches:
        enum_name = match.group(1)
        enum_body = match.group(2)
        mapping = {}
        pairs = re.findall(r'(\w+)\s*=\s*[\'"](.*?)[\'"]', enum_body)
        for key, val in pairs:
            mapping[val] = key
        enums[enum_name] = mapping
    return enums

def get_enum_ref(enums_map, enum_name, value):
    if not value:
        return "undefined"
    if enum_name in enums_map and value in enums_map[enum_name]:
        return f"{enum_name}.{enums_map[enum_name][value]}"
    return f"'{value}'"

enums_map = parse_enums('C:\\Users\\jeffr\\git\\annocalculator\\src\\app\\shared\\game\\enums.ts')

with open('C:\\Users\\jeffr\\git\\annocalculator\\tmp.json', 'r', encoding="utf8") as f:
    data = json.load(f)

output = []
output.append("import { Item, Rarity, AdministrativeBuilding, ProductionBuilding, Good } from './enums';\n")
output.append("export const items = [")

for entry in data:
    output.append("  {")
    item_val = entry.get('item')
    item_ref = get_enum_ref(enums_map, 'Item', item_val)
    output.append(f"    item: {item_ref},")
    output.append(f"    iconUrl: '{entry.get('iconUrl', '')}',")
    output.append(f"    rarity: {get_enum_ref(enums_map, 'Rarity', entry.get('rarity'))},")
    output.append(f"    administrativeBuilding: {get_enum_ref(enums_map, 'AdministrativeBuilding', entry.get('administrativeBuilding'))},")
    targets = entry.get('targets', [])
    target_refs = [get_enum_ref(enums_map, 'ProductionBuilding', t) for t in targets]
    output.append(f"    targets: [{', '.join(target_refs)}],")
    if 'extraGoods' in entry and entry['extraGoods']:
        output.append("    extraGoods: [")
        for eg in entry['extraGoods']:
            output.append(f"      {{ good: {get_enum_ref(enums_map, 'Good', eg.get('good'))}, rateNumerator: {eg.get('rateNumerator')}, rateDenominator: {eg.get('rateDenominator')} }},")
        output.append("    ],")
    if 'replacementGoods' in entry and entry['replacementGoods']:
        output.append("    replacementGoods: [")
        for rg in entry['replacementGoods']:
            output.append(f"      {{ from: {get_enum_ref(enums_map, 'Good', rg.get('from'))}, to: {get_enum_ref(enums_map, 'Good', rg.get('to'))} }},")
        output.append("    ],")
    output.append(f"    productivityEffect: {entry.get('productivityEffect', 0)},")
    output.append(f"    providesElectricity: {str(entry.get('providesElectricity', False)).lower()}")
    output.append("  },")
output.append("];")

final_ts = "\n".join(output)

with open('items.ts', 'w') as f:
    f.write(final_ts)

# Instead of printing everything (too large), I'll just confirm it's written.
# The user needs it in "Files". I will tell them it's in the sandbox.
print(f"File created. Size: {len(final_ts)} bytes.")

File created. Size: 350553 bytes.
